In [1]:
from pprint import pprint

import numpy as np
from matplotlib import pyplot as plt

from GamerVis import *

## Introduction
The main class is `gamervis()`, which inherits from relevant base classes and provides functions for analyzing and visualizing GAMER data from CCSN simulations.

Current supports:
- I/O
    - `Record__Note`
    - `Record__CentralQuant`
    - `Record__QuadMom_2nd`
    - HDF5 snapshots 
- Analysis
    - PNS radius and mass
    - Spherically averaged profiles
    - Accretion rate
    - The onset condition of shock expansion (the ratio of advection to heating timescales)
- Visualization
    - Evolution of quantities in `Record__CentralQuant`
    - Evolution of strain, spectrogram, and amplitude spectral density of GW emissions
    - Slice plot
    - Slice plot of density fluctuation (for low-T/|W| instability)
- Various gadgets

In [2]:
gamervis?

Init signature: gamervis(rundir='.', tbounce=0.0, nuctable=None)
Docstring:      Class for analyzing and visualizing GAMER data for CCSN simulations.
Init docstring:
Parameters
----------
rundir: string, optional
    Path to the simulation data.
tbounce: float, optional
    Physical time of core bounce, in second.
nuctable: string, optional
    Path to the nuclear EoS table.
    If set to "runtime", the value recorded in Record__Note will be used.
File:           ~/opt/python-3.11.9_gcc-11.2.0/lib/python3.11/site-packages/GamerVis/gamervis.py
Type:           type
Subclasses:     

The first step is to instantiate a `gamervis` object. The nuclear EoS solver is still under development and is not required unless you are analyzing MRI.

In [3]:
rundir  = "/home/hfhsieh/nas/ccsn_leakage_prod/B00_Rot00/MHM_AngMax00_AngMin12_B0_Rot0_re4"
tbounce = 4.5865000e-01

gamer_obj = gamervis(rundir = rundir, tbounce = tbounce)

print("Path to simulation data:", gamer_obj.rundir)
print("Bounce time:", gamer_obj.tbounce)

Path to simulation data: /home/hfhsieh/nas/ccsn_leakage_prod/B00_Rot00/MHM_AngMax00_AngMin12_B0_Rot0_re4
Bounce time: 0.45865


## Basic usage

- `obj.get_allhdf5files(path = self.rundir)`
    - Retrieve all HDF5 files in the specified path    

- `obj.extend_filename(fn_or_idx)`
    - Complete the full filename of an HDF5 file (in format of `PATH_TO_FILE/Data_*`).
    - `fn_or_idx` can be integer.

- `obj.get_file_index(fn)`
    - Retrieve the file index.

- `obj.get_time(fn, cgs = True)`
    - Get the physical time recorded in the specified HDF5 file.
    
- `obj.get_energyshift(fn = None)`
    - Get the energy shift used in the adopted nuclear EoS table.
    - If `fn` is not specified, use the values of runtime parameters in `Record__Note`.
    - Current supports `LS220` and `SFHo`.
    
- `obj.get_param(keyword, fn = None, source = "ascii")`
    - Get the value corresponding to the specified `keyword` from various sources.
    - Supports:
        - HDF5 file (specified by `fn` and `source = "hdf5"`).
        - `Record__Note` (`source = "note"`). Case insensitive.
        - `Input__TestProb` (`source = "testprob`). Case sensitive.

- `obj.get_unitsys(fn)`
    - Get the unit system recorded in the specified HDF5 file and store the data in `self.unit` (dict)

- `obj.get_center(fn, center)`
    - Get the coordinates of reference center at the physical time recorded in the specified HDF5 file.
    - Supports:
        - "c": Box center.
        - "pns_ascii": PNS center in `Record__CentralQuant`.
        - "pns_hdf5": Coordinate of highest-density cell in the HDF5 files (search through cells with density higher than $10^{14}$ g/cm$^{3}$).

- `obj.get_centquant()`
    - Load the `Record__CentralQuant` and store the data in `self.centquant` (numpy structured array)

- `obj.get_quadmom()`
    - Load the `Record__QuadMom_2nd` and store the data in `self.quadmom` (numpy structured array)

- `obj.interp_centquant(field, fn = None, time = None)`
    - Interpolate the data in `Record__CentralQuant` to obtain the coressponding value at the specified time.
    - The physical time can be specified either using `time` or by retrieving if from the time recorded in the specified HDF5 file.

In [4]:
### get_allhdf5files()
fn_list = gamer_obj.get_allhdf5files()

pprint(fn_list[:5])

['/home/hfhsieh/nas/ccsn_leakage_prod/B00_Rot00/MHM_AngMax00_AngMin12_B0_Rot0_re4/Data_000124',
 '/home/hfhsieh/nas/ccsn_leakage_prod/B00_Rot00/MHM_AngMax00_AngMin12_B0_Rot0_re4/Data_000125',
 '/home/hfhsieh/nas/ccsn_leakage_prod/B00_Rot00/MHM_AngMax00_AngMin12_B0_Rot0_re4/Data_000126',
 '/home/hfhsieh/nas/ccsn_leakage_prod/B00_Rot00/MHM_AngMax00_AngMin12_B0_Rot0_re4/Data_000127',
 '/home/hfhsieh/nas/ccsn_leakage_prod/B00_Rot00/MHM_AngMax00_AngMin12_B0_Rot0_re4/Data_000128']


In [5]:
### extend_filename()
fn_tmp = gamer_obj.extend_filename(1)

pprint(fn_tmp)

'/home/hfhsieh/nas/ccsn_leakage_prod/B00_Rot00/MHM_AngMax00_AngMin12_B0_Rot0_re4/Data_000001'


In [6]:
### get_file_index()
gamer_obj.get_file_index("FOOBAR/Data_123456")

123456

In [7]:
### get_time()
fn_tmp = gamer_obj.extend_filename(130)

print("Code Unit:", gamer_obj.get_time(fn_tmp, cgs = False))
print("CGS  Unit:", gamer_obj.get_time(fn_tmp))

Code Unit: 698.0
CGS  Unit: 0.6980000000000001


In [8]:
### get_energyshift()
fn_tmp = gamer_obj.extend_filename(130)

print(gamer_obj.get_energyshift(fn_tmp))

yt : [INFO     ] 2025-04-22 06:23:29,933 Parameters: current_time              = 698.0
yt : [INFO     ] 2025-04-22 06:23:29,934 Parameters: domain_dimensions         = [160 160 160]
yt : [INFO     ] 2025-04-22 06:23:29,934 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-04-22 06:23:29,935 Parameters: domain_right_edge         = [20000. 20000. 20000.]
yt : [INFO     ] 2025-04-22 06:23:29,935 Parameters: cosmological_simulation   = 0


8.857790475887134e+18


In [9]:
### get_param()
fn_tmp = gamer_obj.extend_filename(130)

print("NUC Table: ", gamer_obj.get_param("NucTable", fn = fn_tmp, source = "hdf5"))
print("BOX Size : ", gamer_obj.get_param("Box_Size", source = "note"))
print("CCSN_Prob: ", gamer_obj.get_param("CCSN_Prob", source = "testprob"))

NUC Table:  /home/hfhsieh/work/EoS/LS220_234r_136t_50y_SVNr26_136e_136s_136p_v1.h5
BOX Size :  20000.0
CCSN_Prob:  1


In [10]:
### get_unitsys()
fn = gamer_obj.extend_filename(130)

gamer_obj.get_unitsys(fn)

print("Stamp:", gamer_obj.unit_stamp, "\n")
pprint(gamer_obj.unit)

Stamp: 130 

{'B': 4.999451085065279e+17,
 'D': 1.989e+18,
 'E': 1.9890000000000002e+49,
 'L': 100000.0,
 'M': 1.989e+33,
 'P': 1.989e+34,
 'T': 0.001,
 'V': 100000000.0}


In [11]:
### get_center()
fn = gamer_obj.extend_filename(130)

print("Box center                        :", gamer_obj.get_center(fn, "c"))
print("PNS center in Record__CentralQuant:", gamer_obj.get_center(fn, "pns_ascii"))
print("PNS center in HDF5                :", gamer_obj.get_center(fn, "pns_hdf5"))

yt : [INFO     ] 2025-04-22 06:23:33,179 Parameters: current_time              = 698.0
yt : [INFO     ] 2025-04-22 06:23:33,180 Parameters: domain_dimensions         = [160 160 160]
yt : [INFO     ] 2025-04-22 06:23:33,181 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-04-22 06:23:33,181 Parameters: domain_right_edge         = [20000. 20000. 20000.]
yt : [INFO     ] 2025-04-22 06:23:33,182 Parameters: cosmological_simulation   = 0


Box center                        : [1.e+09 1.e+09 1.e+09] cm
PNS center in Record__CentralQuant: [1.e+09 1.e+09 1.e+09] cm


yt : [INFO     ] 2025-04-22 06:23:37,604 Parameters: current_time              = 698.0
yt : [INFO     ] 2025-04-22 06:23:37,605 Parameters: domain_dimensions         = [160 160 160]
yt : [INFO     ] 2025-04-22 06:23:37,605 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-04-22 06:23:37,606 Parameters: domain_right_edge         = [20000. 20000. 20000.]
yt : [INFO     ] 2025-04-22 06:23:37,606 Parameters: cosmological_simulation   = 0
yt : [INFO     ] 2025-04-22 06:24:19,000 max value is 5.77777e+14 at 10000.1220703125000000 9999.8779296875000000 10000.1220703125000000


PNS center in HDF5                : [1.00001221e+09 9.99987793e+08 1.00001221e+09] cm


In [12]:
### interp_centquant()
fn   = gamer_obj.extend_filename(130)
time = gamer_obj.get_time(fn)

print(gamer_obj.interp_centquant("dens", time = time))
print(gamer_obj.interp_centquant("dens", fn = fn))

print("\nAvailable field name: ")
pprint(gamer_obj.centquant.dtype.names)

577777000000000.0
577777000000000.0

Available field name: 
('time',
 'step',
 'posx',
 'posy',
 'posz',
 'dens',
 'ye',
 'rsh_min',
 'rsh_ave_V',
 'rsh_ave_Vinv',
 'rsh_max',
 'ccsn_x',
 'ccsn_y',
 'ccsn_z',
 'leak_netheat',
 'leak_lum_nue',
 'leak_lum_nua',
 'leak_lum_nux',
 'leak_heat_nue',
 'leak_heat_nua',
 'leak_netheat_nue',
 'leak_netheat_nua',
 'leak_eave_nue',
 'leak_eave_nua',
 'leak_eave_nux',
 'leak_radns_nue',
 'leak_radns_nua',
 'leak_radns_nux')


In [13]:
### get_centquant()
gamer_obj.get_centquant()

data = gamer_obj.centquant

# print all data
pprint(data)

# can use name to retrieve data of the specified field
print("\nRsh_Vinv:\n", data["rsh_ave_V"])

array([(0.0000000e+00,    0, 1.0000488e+09, 9.9838867e+08, 9.9663086e+08, 2.1697799e+09, 0.44694963,       0. ,        0.,        0.,        0., 1.e+09, 1.e+09, 1.e+09, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,  0.      ,  0.      ,  0.      ,       0. ,       0. ,       0. ),
       (1.4040356e-07,    1, 1.0000488e+09, 9.9995117e+08, 9.9995117e+08, 2.1698081e+09, 0.44695035,       0. ,        0.,        0.,        0., 1.e+09, 1.e+09, 1.e+09, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,  0.      ,  0.      ,  0.      ,       0. ,       0. ,       0. ),
       (1.0014040e-04,    2, 9.9995117e+08, 1.0000488e+09, 1.0000488e+09, 2.1795879e+09, 0.44695032,       0. ,        0.,        0.,        0., 1.e+09, 1.e+09, 1.e+09, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,